
MMDetection tutorial을 활용하여 베이스라인을 만들었습니다.

coco format으로 변환하는 부분은 이여름님께서 공유해주신 [코드](https://dacon.io/competitions/official/235855/codeshare/3729)를 참고했고

데이터 정제 과정과 추론 부분은 데이콘에서 제공해 주신 [베이스라인](https://dacon.io/competitions/official/235855/codeshare/3725)을 참고했습니다.


**참고 링크** <p>
- [이여름님의 코드](https://dacon.io/competitions/official/235855/codeshare/3729)
- [데이콘 베이스라인](https://dacon.io/competitions/official/235855/codeshare/3725)
- [colab version tutorial](https://github.com/open-mmlab/mmdetection/blob/master/demo/MMDet_Tutorial.ipynb)
- [kaggle notebook](https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-l-cascadercnn-mmdetection-infer?scriptVersionId=68887943&cellId=21)


**MMDetection 관련 링크**
- [mmdetection github](https://github.com/open-mmlab/mmdetection)
- [mmdetection docs](https://mmdetection.readthedocs.io/)
- [model_zoo](https://mmdetection.readthedocs.io/en/latest/model_zoo.html#baselines)

*❗❗* 코드는 모든 학습 데이터를 사용할 수 있도록 해놓았지만 출력은 일부 데이터만 사용한 결과물이며 로컬 환경과 코랩 환경을 오가며 진행했습니다.

## Prepare data

In [ ]:
# basic setup
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import cv2
import base64
import time
import math
import datetime
import os
import zipfile
import random
from PIL import Image
from io import BytesIO
from tqdm.notebook import tqdm
from glob import glob

from collections import defaultdict

base_dir = "/content/project/input"


In [ ]:
os.makedirs('./project/input', exist_ok=True)
os.makedirs('./project/output', exist_ok=True)
%cd project

/content/project


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# https://drive.google.com/file/d/1SrazHXmnJwL4_qBV1EDiVN60mWUMEATn/view?usp=sharing

# !gdown 1oQdnhO7uHgiZKAUY8UJdpk5ig479CZug
with zipfile.ZipFile('/content/drive/MyDrive/project/vinBigData/Chest X-ray(512).zip', 'r') as zip_ref:
  zip_ref.extractall('/content/project/input')
#!rm './Chest X-ray(512).zip'

In [ ]:
%pwd

'/content/project'

In [ ]:
df = pd.read_csv('./input/train.csv')
df = df[df['class_id'] != 14]
df['class_id'] = df['class_id'] + 1
df = df[['image_id', 'class_id', 'x_min', 'y_min', 'x_max', 'y_max']]
df

,image_id,class_id,x_min,y_min,x_max,y_max
6,9a5094b2563a1ef3ff50dc5c7ff71345,4,170.092308,301.369863,406.892308,401.315068
7,9a5094b2563a1ef3ff50dc5c7ff71345,11,440.369231,378.958904,461.538462,436.602740
8,9a5094b2563a1ef3ff50dc5c7ff71345,12,440.369231,378.958904,461.538462,436.602740
9,9a5094b2563a1ef3ff50dc5c7ff71345,4,170.338462,301.369863,407.876923,394.301370
10,9a5094b2563a1ef3ff50dc5c7ff71345,4,169.600000,287.780822,410.092308,386.410959
...,...,...,...,...,...,...
67774,52951d7de2485aba8ed62629eee4d254,10,67.333333,256.355556,85.111111,268.088889
67775,52951d7de2485aba8ed62629eee4d254,4,163.111111,279.288889,358.666667,336.355556
67848,1224f07d895107573588225f692e94f9,1,250.729412,161.922261,320.250980,223.434629
67849,1224f07d895107573588225f692e94f9,1,262.525490,155.590106,319.247059,221.399293


==========

convert_to_coco 에서 train image도 jpg로 저장하도록 했습니다.

In [ ]:
def convert_to_coco(name_list, df, save_path): # 변경

  res = defaultdict(list)
    
  categories = {
      'Aortic enlargement': 1,
      'Atelectasis': 2,
      'Calcification': 3,
      'Cardiomegaly': 4,
      'Consolidation': 5,
      'ILD': 6,
      'Infiltration': 7,
      'Lung Opacity': 8,
      'Nodule/Mass': 9,
      'Other lesion': 10,
      'Pleural effusion': 11,
      'Pleural thickening': 12,
      'Pneumothorax': 13,
      'Pulmonary fibrosis': 14
  }
    
  df = df[df['image_id'].isin(name_list)]
  names = df['image_id'].unique()
  n_id = 0

  for pic_name in tqdm(names):

    df_temp = df[df['image_id'] == pic_name]
    tmp = df_temp.values

    res['images'].append({
        'id': pic_name,
        'width': 512,
        'height':512,
        'file_name': pic_name+".png"
    })
      
    for shape in tmp:
        x1, y1, x2, y2 = shape[2], shape[3], shape[4], shape[5]
        
        w, h = x2 - x1, y2 - y1
        
        res['annotations'].append({
            'id': n_id,
            'image_id': pic_name,
            'category_id': shape[1],
            'area': w * h,
            'bbox': [x1, y1, w, h],
            'iscrowd': 0,
        })
        n_id += 1
    
  for name, id in categories.items():
      res['categories'].append({
          'id': id,
          'name': name,
      })
  # return res
  with open(save_path, 'w') as f:
      json.dump(res, f)

In [ ]:
random.seed(10)

# train_files = glob(os.path.join(base_dir, 'train/*.png'))
# train_files = os.listdir('./input/train')
# train_files = list(map(del_extension, train_files))
train_files = df['image_id'].unique()

random.shuffle(train_files)

# 8:2로 학습/검증 데이터 분리
split_num = int(len(train_files)*0.2)
print("split_num :", split_num)

train_file = train_files[split_num:]
valid_file = train_files[:split_num]

len(train_file), len(valid_file),len(train_files)

split_num : 878


(3516, 878, 4394)

In [ ]:
convert_to_coco(train_file, df, os.path.join(base_dir, 'train_annotations.json'))

  0%|          | 0/3516 [00:00<?, ?it/s]

In [ ]:
convert_to_coco(valid_file, df, os.path.join(base_dir, 'valid_annotations.json'))

  0%|          | 0/878 [00:00<?, ?it/s]

## Environment setting

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
!pip show torch

Name: torch
Version: 1.12.1+cu113
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /usr/local/lib/python3.7/dist-packages
Requires: typing-extensions
Required-by: torchvision, torchtext, torchaudio, fastai


런타임 다시 시작하고 진행합니다.

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git

Cloning into 'mmdetection'...
remote: Enumerating objects: 32062, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (165/165), done.
remote: Total 32062 (delta 63), reused 91 (delta 21), pack-reused 31875
Receiving objects: 100% (32062/32062), 41.10 MiB | 25.16 MiB/s, done.
Resolving deltas: 100% (23025/23025), done.


In [ ]:
%cd ./mmdetection

/content/project/mmdetection


In [ ]:
!pip install openmim
!mim install mmdet==2.22.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 3.8 MB/s 
     |████████████████████████████████| 237 kB 13.7 MB/s 
     |████████████████████████████████| 51 kB 7.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu113/torch1.12.0/index.html
     |████████████████████████████████| 1.3 MB 7.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 728 kB 7.1 MB/s 
Saved /tmp/tmpwjgkngv1/mmdet-2.22.0.tar.gz
Successfully downloaded mmdet
Get 'mim' extra requirements from `mminstall.txt` for mmdet 2.22.0: ['mmcv-full>=1.3.17'].
     |████████████████████████████████| 40.6 MB 9.4 MB/s 
     |████████████████████████████████| 190 kB 82.7 MB/s 
Using cached `mminstall.txt` for mmdet==2.22.0: /ro

=========

In [ ]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.12.1+cu113 True
2.25.2
11.3
GCC 9.3


tutorial에 있는 mask_rcnn 모델을 그대로 사용해도 되지만 저는 faster_rcnn 모델을 사용했습니다.

In [ ]:
# %cd mmdetection
# !mkdir checkpoints
# !wget -c https://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth \
#       -O checkpoints/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

# %cd mmdetection
!mkdir checkpoints
!wget -c https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth \
      -O checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth

--2022-10-21 02:25:25--  https://download.openmmlab.com/mmdetection/v2.0/faster_rcnn/faster_rcnn_r50_fpn_1x_coco/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.88.36.86
Connecting to download.openmmlab.com (download.openmmlab.com)|47.88.36.86|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167287506 (160M) [application/octet-stream]
Saving to: ‘checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’

checkpoints/faster_ 100%[===================>] 159.54M  11.8MB/s    in 12s     

2022-10-21 02:25:38 (13.2 MB/s) - ‘checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth’ saved [167287506/167287506]



In [ ]:
# Choose to use a config and initialize the detector
config = "configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py"
# Setup a checkpoint file to load
checkpoint = '/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
# initialize the detector
#model = init_detector(config, checkpoint, device='cuda:0')

## Train a detector on customized dataset

To train a new detector, there are usually three things to do:
1. Support a new dataset
2. Modify the config
3. Train a new detector



In [ ]:
import mmcv
import matplotlib.pyplot as plt

import copy
import os.path as osp

import mmcv
import numpy as np

### Modify the config

In the next step, we need to modify the config for the training.
To accelerate the process, we finetune a detector using a pre-trained detector.

In [ ]:
from mmcv import Config
cfg = Config.fromfile(config)

Given a config that trains a Faster R-CNN on COCO dataset, we need to modify some values to use it for training Faster R-CNN on KITTI dataset.

COCO datasets과 같은 standard datasets에서 미리 정의된 모델(config) 에서 학습하는 방법을 보여 줌

 

config 파일의 기본 learning rate는 GPU 8개와 2 img/gpu (batch size = 8 * 2 = 16)

 

Linear scaling 규칙에 따라 GPU당 다른 GPU 또는 이미지를 사용하는 경우 

배치 크기에 비례하여 학습률을 설정.

 

lr=0.01 for   4 GPUs * 2 imgs/gpu
lr=0.08 for 16 GPUs * 4 imgs/gpu




resume-from 과 load-from의 차이

resume-from : weight와 optimizer 상태 모두 로드 및 epoch도 지정된 checkpoint에서 상속

(일반적으로 학습하다가 중단되었을  때)

 

load-from : weight만 load 하고 training epoch는 0부터 시작

(일반적으로 fine-tuning 할 때)

 

위를 참고하여 아래의 명령어를 실행하여 faster rcnn COCO dataset 학습을 진행하였다.




In [ ]:
from mmdet.apis import set_random_seed

base_path = '/content/project/input' # base_dir과 같습니다.

# Set up working dir to save files and logs.
cfg.work_dir = '/content/drive/MyDrive/project/vinBigData/faster/2'

EPOCHS= 30




train_anno = "train_annotations.json" # 출력은 "train_partial_annotations.json"
valid_anno = "valid_annotations.json" # 출력은 "valid_partial_annotations.json"
test_anno = "valid_annotations.json" # 출력은 "valid_partial_annotations.json"

train_img = "train" # 출력은 "train_100000"
valid_img = 'train'
test_img =  "test" # 출력은 "train_100000"



# Modify dataset type and path
cfg.dataset_type = 'CocoDataset'
cfg.data_root = base_path

#train
cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = base_path
cfg.data.train.ann_file = train_anno
cfg.data.train.img_prefix = train_img

#valid
cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = base_path
cfg.data.val.ann_file = valid_anno
cfg.data.val.img_prefix = valid_img

#test
cfg.data.test.type = 'CocoDataset'
cfg.data.test.data_root = base_path
cfg.data.test.ann_file = test_anno
cfg.data.test.img_prefix = test_img



cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 2

classes = (
    'Aortic enlargement',
    'Atelectasis',
    'Calcification',
    'Cardiomegaly',
    'Consolidation',
    'ILD',
    'Infiltration',
    'Lung Opacity',
    'Nodule/Mass',
    'Other lesion',
    'Pleural effusion',
    'Pleural thickening',
    'Pneumothorax',
    'Pulmonary fibrosis'
    )

cfg.data.train.classes = classes
cfg.data.val.classes = classes
cfg.data.test.classes = classes


# modify num classes of the model in box head
cfg.model.roi_head.bbox_head.num_classes = 14
# We can still use the pre-trained Mask RCNN model though we do not need to
# use the mask branch

#load-from : weight만 load 하고 training epoch
# 사전 학습 모델 사용 (fine-tunning)
cfg.load_from = "/content/project/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth"

#resume-from : weight와 optimizer 상태 모두 로드 및 epoch도 지정된 checkpoint에서 상속
cfg.resume_from = None

# The original learning rate (LR) is set for 8-GPU training.
# We divide it by 8 since we only use one GPU.
cfg.optimizer = dict(type='Adam', lr=0.0003, weight_decay=0.0001)
cfg.optimizer.lr = 0.02 / 8
cfg.lr_config.warmup = None

# 에폭 수 조절
# the max_epochs and step in lr_config need specifically tuned for the customized dataset
cfg.runner = dict(type='EpochBasedRunner', max_epochs=EPOCHS)
cfg.log_config.interval = 10

# Change the evaluation metric since we use customized dataset.
cfg.evaluation.metric = 'bbox'

# We can set the evaluation interval to reduce the evaluation times
cfg.evaluation.interval = 4

# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 4

# Set seed thus the results are more reproducible
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

cfg.device='cuda'

# We can initialize the logger for training and have a look
# at the final config used for training
print(f'Config:\n{cfg.pretty_text}')


Config:
model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_w

### Train a new detector

Finally, lets initialize the dataset and detector, then train a new detector!

In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))


loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [ ]:
t_files = glob(os.path.join('/content/project/input/train/*.png'))
v_files = glob(os.path.join('/content/project/input/test/*.png'))
len(t_files),len(v_files)

(15000, 3000)

In [ ]:
print(datasets)

[
CocoDataset Train dataset with number of images 3516, and instance counts: 
+------------------------+-------+-------------------------+-------+-------------------+-------+-------------------------+-------+-------------------+-------+
| category               | count | category                | count | category          | count | category                | count | category          | count |
+------------------------+-------+-------------------------+-------+-------------------+-------+-------------------------+-------+-------------------+-------+
| 0 [Aortic enlargement] | 5793  | 1 [Atelectasis]         | 209   | 2 [Calcification] | 808   | 3 [Cardiomegaly]        | 4357  | 4 [Consolidation] | 441   |
| 5 [ILD]                | 806   | 6 [Infiltration]        | 1042  | 7 [Lung Opacity]  | 1980  | 8 [Nodule/Mass]         | 2058  | 9 [Other lesion]  | 1742  |
| 10 [Pleural effusion]  | 1921  | 11 [Pleural thickening] | 3834  | 12 [Pneumothorax] | 186   | 13 [Pulmonary fibrosis] | 3689

In [ ]:
# train!!!
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-10-21 02:26:44,290 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-10-21 02:26:44,335 - mmdet - INFO - load checkpoint from local path: /content/project/mmdetection/checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-10-21 02:26:44,464 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([15, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([15]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([56, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in curren

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


2022-10-21 02:27:06,440 - mmdet - INFO - Epoch [1][10/440]	lr: 2.500e-03, eta: 2:40:35, time: 2.195, data_time: 0.269, memory: 9886, loss_rpn_cls: 569236.7642, loss_rpn_bbox: 74292.1795, loss_cls: 18077879.8164, acc: 23.4004, loss_bbox: 3441753.1321, loss: 22163160.9359
2022-10-21 02:27:19,491 - mmdet - INFO - Epoch [1][20/440]	lr: 2.500e-03, eta: 2:07:44, time: 1.305, data_time: 0.018, memory: 9886, loss_rpn_cls: 8660.8913, loss_rpn_bbox: 390.1924, loss_cls: 5623.5648, acc: 7.3352, loss_bbox: 8812.1449, loss: 23486.7935
2022-10-21 02:27:32,677 - mmdet - INFO - Epoch [1][30/440]	lr: 2.500e-03, eta: 1:56:58, time: 1.319, data_time: 0.018, memory: 9886, loss_rpn_cls: 147.5965, loss_rpn_bbox: 67.3339, loss_cls: 542.7744, acc: 0.5124, loss_bbox: 178.8570, loss: 936.5617
2022-10-21 02:27:46,483 - mmdet - INFO - Epoch [1][40/440]	lr: 2.500e-03, eta: 1:52:37, time: 1.381, data_time: 0.017, memory: 9886, loss_rpn_cls: 157.9862, loss_rpn_bbox: 24.4051, loss_cls: 289.5069, acc: 79.0947, loss_bbo

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 878/878, 11.4 task/s, elapsed: 77s, ETA:     0s

2022-10-21 02:38:50,805 - mmdet - INFO - Evaluating bbox...


...

DONE (t=0.80s).
Accumulating evaluation results...
DONE (t=0.17s).


2022-10-21 04:05:17,784 - mmdet - INFO - Epoch [10][10/440]	lr: 2.500e-04, eta: 0:09:35, time: 1.569, data_time: 0.266, memory: 9887, loss_rpn_cls: 889.5548, loss_rpn_bbox: 295.0257, loss_cls: 2.2860, acc: 23.1563, loss_bbox: 0.0003, loss: 1186.8667
2022-10-21 04:05:30,935 - mmdet - INFO - Epoch [10][20/440]	lr: 2.500e-04, eta: 0:09:22, time: 1.315, data_time: 0.018, memory: 9887, loss_rpn_cls: 841.6066, loss_rpn_bbox: 304.0751, loss_cls: 2.3359, acc: 20.0745, loss_bbox: 0.0003, loss: 1148.0178
2022-10-21 04:05:44,170 - mmdet - INFO - Epoch [10][30/440]	lr: 2.500e-04, eta: 0:09:09, time: 1.324, data_time: 0.019, memory: 9887, loss_rpn_cls: 844.2323, loss_rpn_bbox: 333.5124, loss_cls: 2.2189, acc: 23.2338, loss_bbox: 0.0002, loss: 1179.9638
2022-10-21 04:05:57,387 - mmdet - INFO - Epoch [10][40/440]	lr: 2.500e-04, eta: 0:08:55, time: 1.322, data_time: 0.020, memory: 9887, loss_rpn_cls: 931.7098, loss_rpn_bbox: 308.8464, loss_cls: 2.3595, acc: 20.2973, loss_bbox: 0.0002, loss: 1242.9159


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 878/878, 13.9 task/s, elapsed: 63s, ETA:     0s

2022-10-21 04:15:50,245 - mmdet - INFO - Evaluating bbox...
2022-10-21 04:15:50,247 - mmdet - ERROR - The testing results of the whole dataset is empty.
ERROR:mmdet:The testing results of the whole dataset is empty.
2022-10-21 04:15:50,251 - mmdet - INFO - Epoch(val) [10][878]	


Loading and preparing results...


# 학습 이어서 하기!

In [ ]:
# model로는 이어서 안됨, test 추론 할때 사용
# torch.save(model,'/content/drive/MyDrive/vinBigData/faster/model.pt')
# checkpoint로 이어서 가능
#resume-from : weight와 optimizer 상태 모두 로드 및 epoch도 지정된 checkpoint에서 상속
cfg.resume_from = '/content/drive/MyDrive/vinBigData/faster/2/latest.pth'
cfg.load_from = None


In [ ]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector


# Build dataset
datasets = [build_dataset(cfg.data.train)]
print(datasets)

# Build the detector
model = build_detector(
    cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

# train!!!
train_detector(model, datasets, cfg, distributed=False, validate=True)

# model로는 이어서 안됨, test 추론 할때 사용
torch.save(model,'/content/drive/MyDrive/vinBigData/faster/model.pt')

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!
[
CocoDataset Train dataset with number of images 3516, and instance counts: 
+------------------------+-------+-------------------------+-------+-------------------+-------+-------------------------+-------+-------------------+-------+
| category               | count | category                | count | category          | count | category                | count | category          | count |
+------------------------+-------+-------------------------+-------+-------------------+-------+-------------------------+-------+-------------------+-------+
| 0 [Aortic enlargement] | 5793  | 1 [Atelectasis]         | 209   | 2 [Calcification] | 808   | 3 [Cardiomegaly]        | 4357  | 4 [Consolidation] | 441   |
| 5 [ILD]                | 806   | 6 [Infiltration]        | 1042  | 7 [Lung Opacity]  | 1980  | 8 [Nodule/Mass]         | 2058  | 9 [Other lesion]  | 1742  |
| 10 [Pleural effusion]  | 1921  | 11 [Pleur

2022-10-21 04:18:11,167 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-10-21 04:18:11,205 - mmdet - INFO - load checkpoint from local path: /content/drive/MyDrive/vinBigData/faster/2/latest.pth


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


2022-10-21 04:18:16,433 - mmdet - INFO - resumed epoch 6, iter 2640
2022-10-21 04:18:16,436 - mmdet - INFO - Start running, host: root@69e6261e5325, work_dir: /content/drive/MyDrive/project/vinBigData/faster/2
2022-10-21 04:18:16,438 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                     

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 878/878, 11.7 task/s, elapsed: 75s, ETA:     0s

2022-10-21 04:40:54,494 - mmdet - INFO - Evaluating bbox...
2022-10-21 04:40:54,498 - mmdet - ERROR - The testing results of the whole dataset is empty.
ERROR:mmdet:The testing results of the whole dataset is empty.
2022-10-21 04:40:54,502 - mmdet - INFO - Epoch(val) [8][878]	


Loading and preparing results...


2022-10-21 04:41:11,655 - mmdet - INFO - Epoch [9][10/440]	lr: 2.500e-04, eta: 3:54:58, time: 1.713, data_time: 0.264, memory: 9891, loss_rpn_cls: 0.3074, loss_rpn_bbox: 0.0633, loss_cls: 0.1103, acc: 98.5400, loss_bbox: 0.0013, loss: 0.4823
2022-10-21 04:41:26,289 - mmdet - INFO - Epoch [9][20/440]	lr: 2.500e-04, eta: 3:54:44, time: 1.463, data_time: 0.017, memory: 9891, loss_rpn_cls: 0.3161, loss_rpn_bbox: 0.0665, loss_cls: 0.1161, acc: 98.4424, loss_bbox: 0.0012, loss: 0.4999
2022-10-21 04:41:40,952 - mmdet - INFO - Epoch [9][30/440]	lr: 2.500e-04, eta: 3:54:30, time: 1.466, data_time: 0.018, memory: 9891, loss_rpn_cls: 0.3362, loss_rpn_bbox: 0.0774, loss_cls: 0.1285, acc: 98.2324, loss_bbox: 0.0014, loss: 0.5435
2022-10-21 04:41:55,627 - mmdet - INFO - Epoch [9][40/440]	lr: 2.500e-04, eta: 3:54:16, time: 1.467, data_time: 0.016, memory: 9891, loss_rpn_cls: 0.3111, loss_rpn_bbox: 0.0648, loss_cls: 0.1132, acc: 98.4790, loss_bbox: 0.0010, loss: 0.4901
2022-10-21 04:42:10,271 - mmdet 

In [ ]:
# model로는 이어서 안됨, test 추론 할때 사용
torch.save(model,'/content/drive/MyDrive/vinBigData/faster/model.pt')

## Test the trained detector

In [ ]:
from mmdet.apis import inference_detector, show_result_pyplot

import os
from glob import glob
from tqdm.notebook import tqdm
test_img = os.path.join(base_path, "test")

test_file = glob(test_img+"/*.png")

In [ ]:
len(test_file), test_file[0]

테스트 데이터 하나를 추론해봅니다.

In [ ]:
# 추론시 이어서 하기 model 불러오기!
model_test = torch.load('/content/project/output/model.pt')

In [ ]:
img = mmcv.imread(test_file[844])

model_test.cfg = cfg
predictions = inference_detector(model_test, img)
show_result_pyplot(model_test, img, predictions)
print(predictions)

In [ ]:
len(predictions), predictions[0].shape, predictions[1].shape, predictions[2].shape, predictions[3].shape

모든 테스트 데이터에 대해 추론을 진행합니다.

In [ ]:
def format_prediction_string(labels, boxes, scores):
    pred_strings = []
    for j in zip(labels, scores, boxes):
        pred_strings.append("{0} {1:.4f} {2} {3} {4} {5}".format(
            j[0], j[1], j[2][0], j[2][1], j[2][2], j[2][3]))
        # labels scores box_xmin  box_xmax box_ymin box_ymax     
    return " ".join(pred_strings)

In [ ]:
# Ref : https://www.kaggle.com/sreevishnudamodaran/siim-effnetv2-l-cascadercnn-mmdetection-infer?scriptVersionId=68887943&cellId=21


score_threshold = 0.4 # 0.8, 0.3, cfg.model.test_cfg.rcnn.score_thr
results = []


for index, img_path in tqdm(enumerate(test_file), total = len(test_file)):
        
    image_id = img_path.split("/")[-1].split(".")[0]
    file_name = img_path.split("/")[-1].split(".")[0]+".png"
    result = {
        'image_id': image_id,
        'PredictionString': '14 1.0 0 0 1 1'
    }

    img = mmcv.imread(img_path)
    predictions = inference_detector(model, img)

    boxes, scores, labels = (list(), list(), list())

    for k, cls_result in enumerate(predictions):
        # print("cls_result", cls_result)
        if cls_result.size != 0:
            if len(labels)==0:
                boxes = np.array(cls_result[:, :4])
                scores = np.array(cls_result[:, 4])
                labels = np.array([k+1]*len(cls_result[:, 4]))
            else:    
                boxes = np.concatenate((boxes, np.array(cls_result[:, :4])))
                scores = np.concatenate((scores, np.array(cls_result[:, 4])))
                labels = np.concatenate((labels, [k+1]*len(cls_result[:, 4])))

    if len(labels) != 0:
        # 라벨 -1 씩 SHIFT
        labels = labels - 1
        # no finding 이 -1에서 14로 이동!
        labels[labels == -1] = 14

        indexes = np.where(scores > score_threshold)
        # print(indexes)
        
        # 512절대좌표
        boxes = boxes[indexes]


        # 512 상대좌표
        boxes_relative=boxes/512.0
        
        # test 해상도 width, height
        test = test_df[test_df['image_id'] == image_id]
        test_width = float(test['width'])
        test_height = float(test['height'])

        # xmin ymin xmax ymax
        # x * width
        # y * height
        for i,box in enumerate(boxes_relative):
            box[0] = float(box[0] * test_width)
            box[1] = float(box[1] * test_height) 
            box[2] = float(box[2] * test_width)
            box[3] = float(box[3] * test_height)

        # 최종 좌표
        boxes = boxes_relative
        scores = scores[indexes]
        labels = labels[indexes]
        
        # 0.5 보다 confidence 높은 박스가 있다는 의미
        if len(boxes) > 0:
          result = {
              'image_id': image_id,
              'PredictionString': format_prediction_string(labels, boxes, scores)
          }
    # result list를 계속 append!
    results.append(result)


##Inference

In [ ]:
import pandas as pd
submission = pd.DataFrame(results)
submission.shape

In [ ]:
submission.to_csv('/content/drive/MyDrive/vinBigData/submission/mmdedtection_baseline_faster.csv', index=False)